<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/leogabac-Lab4/Lab4/leogabac_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 4: Approximation methods**
**Leonardo Gabriel Alanis Cantú**

# **Abstract**

In this report we will take a look at local polynomial interpolation of a function. The goal is to implement in code the fundamental ideas, and test it. The results were good pretty much everywhere, except for the left enpoint. The reasons for this behaviour is unknown, but there are some untested hypothesis that will be discussed in the last section. Additionally, this code was implemented specifically for one test function, the reasoning behind this decision, and its generalization of this case will also be discussed properly in the last section.

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [2]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
from math import *

# **Introduction**

For a continuous fnction, the _polynomial interpolant_ in the  Lagrange basis is given by

$$
    \pi_q f(x) = \sum_{i=0}^q f(x_i)\lambda(x), \quad x \in I,
$$

where the functions $\lambda_i(x)$ are the so-called Lagrange polynomials. For a given _mesh_ $\mathcal{T} = \{ I_k \}$ with subintervals $I_k = [ x_{k-1}, x_k ], ~ k = 1,\ldots, m+1$ having $m$ internal nodes with not necessarily equal spacing $h_k$, we define the _local_ subinterval vector space as the interpolation of the nodes. The basis for this subspace, is given by the local linear Lagrange shape functions for the subinterval $I_k$.

$$
\lambda_{k,0}(x) = \dfrac{x_k-x}{h_k}, \quad \lambda_{k,1} = \dfrac{x-x_{k-1}}{h_k},
$$

We want to express an interpolant in the global or local basis.

$$
v(x) = \sum_{i=0}^{2(m+1)}\alpha_i\phi_i(x) = \beta_{k,0}\lambda_{k,0} + \beta_{k,1}\lambda_{k,1}.
$$
Since we want to "connect the dots", it is necessary to enforce continuity, this process puts constraints on the degrees of freedom with the mappings $\alpha_k = \beta_{k,1} = \beta_{k+1,0}$ for the internal nodes, and $\alpha_0 = \beta_{1,0}$, and $\alpha_{m+1} = \beta_{m+1,1}$ for the endpoints, note that by doing this, we cut almost in half the amount of degrees of freedom.

For the $L^2$ projection method, it will be of our particular interest to evaluate the polynomials of linear order on each subinterval $I_k$, hence $q=1$ for the implementation of Algorithm 9.1 in the course's textbook.

The $L^2$ projection method relies on the Hilbert space of continuos square integrable functions on the interval $[a,b]$, equipped with the inner product.

$$
(f,g) = \int_a^b f(x)g(x) \text{ d}x,
$$

choosing a particular basis for the prohjection space, it follows that

$$
\sum_{j = 1}^N \alpha_j (\phi_j,\phi_i) = (f,\phi_i),
$$
which is equivalent to a matrix equation
$$
a_{ij} = (\phi_j,\phi_i) = \int_a^b \phi_j(x)\phi_i(x)\text{ d}x,\\
b_i = (f,\phi_i) = \int_a^b f(x) \phi_i(x)\text{ d}x,
$$

From the matrix equation we can compute teh coordinates $\alpha_i$ to get the projection. If the basis supports local shape functions, then we can follow Algorithm 9.1 on the textbook to evaluate the local polynomials.

In this particular work, I will focus on a particular working example(since there will be some computations made by hand), following the process of Example 9.8 in the textbook. 

# **Method**

Consider the function

$$
f(x) = \cos \left( \dfrac{3 \pi}{2}x\right),
$$

which will be represented by

$$
P_h f(x) = \sum_{j=0}^{m+1}\alpha_j \phi_j(x).
$$

Consider as well the equidistant mesh on the interval $[0,1]$ with spacing $h$. Then the matrix $A$ is a tri-diagonal sparse matrix of dimensions $(m+2)\times (m+2)$ elements (Since the index $j= 0,1,2,\ldots,m+1$). For $k = 1,\ldots,m$ we will have the matrix elements

$$
a_{k,k} = \dfrac{2h}{3}, \quad a_{k,k+1} = a_{k,k-1} = \dfrac{h}{6}.
$$

The load vector $b$ is computed as follows.
$$
b_k = \int_0^1 f(x)\phi_k(x) \text{ d}x = \int_{x_{k-1}}^{x_k} f(x)\lambda_{k,1}(x) \text{ d}x + \int_{x_{k}}^{x_{k+1}} f(x)\lambda_{k+1,0}(x) \text{ d}x
$$

Recalling the definitions for $\lambda_{k,j}$ we can perform these integrals (I made them with Wolfram Mathematica), to get

$$
b_k = \dfrac{1}{9h\pi^2}\left[ 8\cos(\omega x_k) - 4\cos(\omega x_{k-1}) - 8\cos(\omega x_{k+1})\right]
$$

There are some tricky things to do on the endpoints, there are several ways to include them:

1. Include an 'virtual/auxiliary' intervals at the endopints that will be used for computation purposes.
2. Use perdiodic boundary conditions (Which is what usually physicists do to ignore the boundaries).
3. Ignore the problem and hope for the best.
4. Go back to the integrals to treat the "extra case".

The proper way to solve this is the fourth option. Following it properly will not change $A$, but the endpoints of the load vector will become (see example 9.7 on textbook p. 197)

$$
b_{0} = \dfrac{1}{9h\pi^2}\left[ 8\cos(\omega x_{0}) - 4\cos(\omega x_{1}) - 6\pi h\sin(\omega x_{0})\right],\\
b_{m+1} = \dfrac{1}{9h\pi^2}\left[ 8\cos(\omega x_{m+1}) - 4\cos(\omega x_{m}) + 6\pi h\sin(\omega x_{m+1})\right].
$$

Again, there are "missing terms", as well as an "extra term". The terms that are missing would have come from the respective vanishing integral, whereas the sine terms are boundary terms that in the "union" would normally disappear.




In [3]:
def computeA(h,m):
    # ===== Input ====== #
    # h: Spacing between nodes 
    # m: Number of internal nodes
    # ===== Output ====== #
    # A: Coefficient matrix A
    
    diag_vec = [2/3 for _ in range(m)] # node terms
    diag_vec.insert(0,2/3) # insert left boundary term
    diag_vec.append(2/3) # insert right boundary term
    
    d = np.diag( diag_vec,0 )
    odp = np.diag( np.zeros(m+1)+1/6,1 )
    odm = np.diag( np.zeros(m+1)+1/6,-1 )
    A = h*(d + odp + odm)
    return A

def bk(xkm,xk,xkp,h, L = False, R = False):
    # ===== Input ====== #
    # xkm: x_{k-1} 
    # xk: x_{k} 
    # xkp: x_{k+1}
    # h: Spacing between nodes
    # endpoint: Boolean, tells you if this is an endpoint
    # ===== Output ===== #
    # b_k : kth element of load vector
    
    w = 3*pi/2
    scale = 1/(9*h*(pi**2))
    if R:
        middle = 8*cos(w*xk) - 4*cos(w*xkm) + 6*pi*h*sin(w*xk)
    elif L:
        middle = 8*cos(w*xk) - 4*cos(w*xkp) - 6*pi*h*sin(w*xk)
    else:
        middle = 8*cos(w*xk) - 4*cos(w*xkm) - 4*cos(w*xkp) 
    
    return scale*middle 

def make_system(mesh):
    # ===== Input ===== #
    # mesh: discretization list with m+2 numbers between 0 and 1
    # ===== Output ===== #
    # A: coefficient matrix
    # b: load vector
    
    h = abs(mesh[1] - mesh[0]) # spacing
    m = len(mesh)-2 # number of internal nodes
    A = computeA(h,m)
    b = [ 0 for _ in range(m+2) ] 
    
    # compute the internal node values for bk
    for k in range(1,m+1):
        b[k] = bk( x[k-1], x[k], x[k+1], h )
    
    b[0] = bk(0, x[0], x[1],h, L = True, R = False)
    b[-1] = bk(x[-2], x[-1], 0,h, L = False, R = True)
    return A,np.array(b)

def get_global_dof(mesh):
    A,b = make_system(mesh)
    Ainv = np.linalg.inv(A)
    return Ainv@b
    

Now that we have gotten the global degrees of freedom, we can actually implement the evaluation of the polynomial. Recall that

$$
v(x) = \sum_{i = 0}^{m+1}\alpha_i \phi_i(x) = \sum_{k=0}^1 \beta_{k,j}\lambda_{k,j},
$$

with $\alpha_k = \beta_k,1 = \beta_{k+1,0}$ for the internal nodes $k = 1,\ldots,m$, and for the endpoints $\alpha_0 = \beta_1,0$, and $\alpha_{m+1} = \beta_{m+1,1}$. Writing this in a way that is more useful for computations we have

$$
(\beta_{k,0}, \beta_{k,1})=\hat{\beta}_k = \begin{cases}
(\alpha_{k-1} , \alpha_k), & \text{for middle points}\\
(\alpha_{0} , 0), & \text{left endpoint} \\
(0 , \alpha_{m+1}), & \text{right endpoint}
\end{cases}
$$

Then we can finally implement Algorithm 9.1 (see textbook page 198).

In [4]:
def lambdak0(mesh,interval,x):
    h = abs( mesh[1] - mesh[0] )
    return ( mesh[interval] - x ) / h

def lambdak1(mesh,interval,x):
    h = abs( mesh[1] - mesh[0] )
    return ( x - mesh[interval-1] ) / h

def get_local_dofs(alpha, interval, M = True, L= False, R = False ):
    
    beta = np.zeros(2)
    
    if L:
        beta[0] = alpha[0]
    elif R:
        beta[1] = alpha[-1]
    else:
        beta[0] = alpha[interval-1]
        beta[1] = alpha[interval]
        
    return beta

def find_interval(mesh,x):
    h = abs(mesh[1] - mesh[0])
    val = x/h
    if x == mesh[0]:
        return 1
    elif x == mesh[-1]:
        return len(mesh) -1
    elif x in mesh:
        return ceil(val)+1
    else:
        return ceil(val)

def eval_local(mesh,interval,x):
    vec = [ lambdak0(mesh,interval,x), lambdak1(mesh,interval,x) ]
    return np.array(vec)
    
def evalpoly(alpha,mesh,x):
    k = find_interval(mesh,x)
    if x == mesh[0]:
        beta = get_local_dofs(alpha,k, M = False, L = True, R = False)
    elif x == mesh[-1]:
        beta = get_local_dofs(alpha,k, M = False, L = False, R = True)
    else:
        beta = get_local_dofs(alpha,k)

    lamb = eval_local(mesh,k,x)
    return np.dot(beta,lamb)

def test_func(x):
    w = (3*pi)/2
    return cos(w*x)
    

# **Results**

We will test the data by comparing different $x$ random values, with the approximation and the exact value.

In [5]:
x = np.linspace(0,1,num = 5)
A,b = make_system(x)
alpha = get_global_dof(x)

In [6]:
n = 10000
xtest = np.random.rand(n)
fexact = np.array( [evalpoly(alpha,x, t) for t in xtest] )
ftest = np.array( [test_func(t) for t in xtest] )

diff = abs(ftest - fexact)
print( "Average difference: ", np.average(diff) )
print( "Median difference: ", np.median(diff) )
print( "Maximum difference: ", np.max(diff) )
print( "Minimum difference: ", np.min(diff) )

Average difference:  0.08741485072796333
Median difference:  0.045706418122078485
Maximum difference:  0.6777757396978094
Minimum difference:  1.4744815242218046e-05


One thing to note, is the defficiencies of my code. The code gives really good approximations everywhere except near the left endpoint. The reason for this is unknown.

In [7]:
approx_nodes = np.array([evalpoly(alpha,x,t) for t in x])
exact_nodes = np.array([test_func(t) for t in x])
diff = abs(exact_nodes-approx_nodes)
print("Difference norm:", np.linalg.norm(diff))
print( "Average difference: ", np.average(diff) )
print( "Median difference: ", np.median(diff) )
print( "Maximum difference: ", np.max(diff) )
print( "Minimum difference: ", np.min(diff) )
print("Error vector:", diff)

Difference norm: 0.6974762266561031
Average difference:  0.19178394034525337
Median difference:  0.10328304087767648
Maximum difference:  0.6780798110042228
Minimum difference:  0.014536567106713328
Error vector: [0.67807981 0.10328304 0.04605346 0.11696682 0.01453657]


I am not sure on why this happens, and unfortunately did not have the time to correct it for the due date.

# **Discussion**

In this report, we covered local piecewise polynomial interpolation to linear order. The results were acceptables, regarding error, however there are still some problems that I would like to discuss.

First, this code is not (yet) general, i.e. it interpolates exactly one test function. The reasoning for this is my own lack of abilities in python. The solution if of course to implement some integration scheme to compute the load vector. Unfortunately this week I had no time to implement a robust Simpson 1/3, and 3/8 method, and my lack of python knowledge, and work this week prevented me to dig into the documentation of new libraries. I have a library of numerical tools in Julia [FiscomTools.jl](https://github.com/leogabac/FiscomTools.jl) which I haven't ported to python projects (yet).

I believe that next week, we will see in class some integration techniques, so I will try to make a better job in the future.

The other problem, which I have no idea on why it arises, is that on the left endpoint, the approximation is considerably worse. My untested hypothesis is that I probably implemented something wrong, and need to do things more carefully.

In general, I believe that the results were acceptable, but there is still a lot of room for improvement.